#### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

1) A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2) A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so the user is asking why parrots talk. I need to make sure I explain this clearly. Let me start by recalling what I know about parrots.\n\nFirst, not all parrots talk, right? I think some species are better at mimicking human speech than others. Maybe the ones that are kept as pets, like African Greys or Amazon parrots, are more vocal. But why do they do it? Is it just mimicry, or is there a purpose?\n\nFrom what I remember, parrots are highly intelligent birds. Their ability to mimic sounds is a form of natural behavior. In the wild, they might mimic other birds or environmental sounds to communicate. So in captivity, when they\'re around humans, they mimic human speech. But is there a social aspect to it? Maybe they want to interact with their owners. I\'ve heard that parrots can form strong bonds with humans, so talking might be a way to engage with their human companions.\n\nAlso, some sources mention that parrots talk to get attention. If they le

In [7]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [8]:
response = model_with_tools.invoke("What's the weather like in Indore?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Indore. I need to use the get_weather function. The function requires a location parameter. Indore is a city in India, so I should check if that\'s the correct location. I don\'t see any other parameters needed, just the location. Let me make sure to format the tool call correctly with the name as get_weather and the arguments including "location": "Indore". That should do it.\n', 'tool_calls': [{'id': 'w9ksyyp6h', 'function': {'arguments': '{"location":"Indore"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 155, 'total_tokens': 273, 'completion_time': 0.197203623, 'completion_tokens_details': {'reasoning_tokens': 93}, 'prompt_time': 0.006624534, 'prompt_tokens_details': None, 'queue_time': 0.058894346, 'total_time': 0.203828157}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_t

### Tool Execution Loops

In [10]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Indore?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Indore is sunny. It's a great day to enjoy outdoor activities! ☀️


In [11]:
messages

[{'role': 'user', 'content': "What's the weather in Indore?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Indore. I need to use the get_weather function. The function requires a location parameter. Indore is a city in India, so I should pass "Indore" as the location. Let me make sure there\'s no typo. Everything looks good. I\'ll call the function with location set to Indore.\n', 'tool_calls': [{'id': 'xmfyg5gwz', 'function': {'arguments': '{"location":"Indore"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 154, 'total_tokens': 253, 'completion_time': 0.146955345, 'completion_tokens_details': {'reasoning_tokens': 74}, 'prompt_time': 0.006994888, 'prompt_tokens_details': None, 'queue_time': 0.053636862, 'total_time': 0.153950233}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason'